### Wake County Parks data cleaning

In [342]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
pd.options.display.width=0

In [369]:
df=pd.read_csv('Parks_in_Wake_County.csv')
original=pd.read_csv('Parks_in_Wake_County.csv')

In [370]:
df.head()

,X,Y,OBJECTID,NAME,ALIAS1,ALIAS2,ADDRESS,Address2,URL,PHONE,ARTSCENTER,BALLFIELDS,BOATRENTAL,CANOE,DISCGOLF,DOGPARK,ENVCTR,FISHING,GREENWAYACCESS,GYM,MULTIPURPOSEFIELD,OUTDOORBASKETBALL,PICNICSHELTER,PLAYGROUND,POOL,COMMUNITYCENTER,NEIGHBORHOODCENTER,TENNISCOURTS,TRACK,WALKINGTRAILS,JURISDICTION,RESTROOMS,Lat,Lon,Notes,ObjectID_1,GlobalID,CreationDate,Creator,EditDate,Editor,AMUSEMENTTRAIN,CAROUSEL,TENNISCENTER,THEATER,BOCCE,HANDBALL,HORSESHOE,INLINESKATING,SANDVOLLEYBALL,SKATEPARK,ACTIVE_ADULT,BMXTRACK,BOATRIDE,LIBRARY,MUSEUM,TEEN,BIKING,LIVEANIMALS,GARDENS,EQUESTRIAN
0,2.135857e+06,735812.904298,1,Anderson Point Park,NaN,NaN,"20 Anderson Point Dr, Raleigh, NC","20 Anderson Point Dr, Raleigh, NC",http://www.raleighnc.gov/arts/content/PRecRecr...,919-329-5994,No,No,No,Yes,No,No,No,No,Yes,No,No,No,Yes,Yes,No,No,No,No,No,Yes,City of Raleigh,Yes,35.770887,-78.542011,NaN,NaN,35b6ff53-86a5-4a9e-9ef2-f04c7cd96978,2017-08-30T20:48:29.798Z,ben.strauss1,2018-03-19T19:05:56.014Z,ben.strauss1,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN
1,2.118778e+06,790779.885547,2,"Annie Louise Wilkerson, MD Nature Preserve Park",NaN,NaN,"11408 Raven Ridge Rd, Raleigh, NC","11408 Raven Ridge Rd, Raleigh, NC",http://www.raleighnc.gov/arts/content/PRecRecr...,919-996-6764,No,No,No,No,No,No,Yes,No,No,No,No,No,Yes,No,No,No,No,No,No,Yes,City of Raleigh,Yes,35.922096,-78.598837,NaN,NaN,d517f738-2730-4903-a5f7-ac93936a5295,2017-08-30T20:48:29.798Z,ben.strauss1,2018-03-19T19:05:54.216Z,ben.strauss1,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN
2,2.115923e+06,733550.256250,3,Apollo Heights Park,Ralph Campbell,NaN,"756 Lunar Dr, Raleigh, NC","756 Lunar Dr, Raleigh, NC",http://www.raleighnc.gov/arts/content/PRecRecr...,919-250-2757,No,No,No,No,No,No,No,No,Yes,No,No,Yes,Yes,Yes,No,Yes,Yes,No,No,No,City of Raleigh,No,35.764905,-78.609241,NaN,NaN,58522d65-c5aa-4b0a-8e20-7d40e211fa4c,2017-08-30T20:48:29.798Z,ben.strauss1,2017-09-22T13:49:58.463Z,ben.strauss1,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN
3,2.098155e+06,785554.030859,4,Baileywick Road Park,NaN,NaN,"9501 Baileywick Rd, Raleigh, NC","9501 Baileywick Rd, Raleigh, NC",http://www.raleighnc.gov/arts/content/PRecRecr...,919-870-2911,No,Yes,No,No,No,No,No,No,Yes,No,Yes,No,Yes,Yes,No,No,No,No,No,Yes,City of Raleigh,Yes,35.907949,-78.668549,NaN,NaN,9177fb9e-cafc-415f-adb0-0bcdfd522717,2017-08-30T20:48:29.798Z,ben.strauss1,2017-09-22T13:49:58.463Z,ben.strauss1,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN
4,2.133114e+06,721092.168750,5,Barwell Road Park,NaN,NaN,"3935 Barwell Rd, Raleigh, NC","3935 Barwell Rd, Raleigh, NC",http://www.raleighnc.gov/arts/content/PRecRecr...,919-329-5994,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,Yes,No,No,No,No,City of Raleigh,No,35.730479,-78.551483,NaN,NaN,4b3305a9-7ab9-4bc0-b397-681cf29feb44,2017-08-30T20:48:29.798Z,ben.strauss1,2017-09-22T13:49:58.463Z,ben.strauss1,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN


We remove columns that are not essential for our porposes and make the table pretty.


In [371]:
drop_columns=['X','Y','OBJECTID','ALIAS1','ALIAS2','Address2','URL','PHONE','Notes','ObjectID_1','GlobalID','CreationDate','Creator','EditDate','Editor']

In [372]:
df.drop(labels=drop_columns,inplace=True,axis=1)
df.head(2)

,NAME,ADDRESS,ARTSCENTER,BALLFIELDS,BOATRENTAL,CANOE,DISCGOLF,DOGPARK,ENVCTR,FISHING,GREENWAYACCESS,GYM,MULTIPURPOSEFIELD,OUTDOORBASKETBALL,PICNICSHELTER,PLAYGROUND,POOL,COMMUNITYCENTER,NEIGHBORHOODCENTER,TENNISCOURTS,TRACK,WALKINGTRAILS,JURISDICTION,RESTROOMS,Lat,Lon,AMUSEMENTTRAIN,CAROUSEL,TENNISCENTER,THEATER,BOCCE,HANDBALL,HORSESHOE,INLINESKATING,SANDVOLLEYBALL,SKATEPARK,ACTIVE_ADULT,BMXTRACK,BOATRIDE,LIBRARY,MUSEUM,TEEN,BIKING,LIVEANIMALS,GARDENS,EQUESTRIAN
0,Anderson Point Park,"20 Anderson Point Dr, Raleigh, NC",No,No,No,Yes,No,No,No,No,Yes,No,No,No,Yes,Yes,No,No,No,No,No,Yes,City of Raleigh,Yes,35.770887,-78.542011,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN
1,"Annie Louise Wilkerson, MD Nature Preserve Park","11408 Raven Ridge Rd, Raleigh, NC",No,No,No,No,No,No,Yes,No,No,No,No,No,Yes,No,No,No,No,No,No,Yes,City of Raleigh,Yes,35.922096,-78.598837,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN


The Longitude and Latitude columns must be moved in front for the table to be clear. The columns will be rearanged

In [373]:
order=list(df.columns)


In [374]:
order.remove('Lat')
order.remove('Lon')

In [375]:
order.insert(2,'Lat')
order.insert(3,'Lon')
order[0:5]

['NAME', 'ADDRESS', 'Lat', 'Lon', 'ARTSCENTER']

In [376]:
df=df[order]
table=df
table.head(2)

,NAME,ADDRESS,Lat,Lon,ARTSCENTER,BALLFIELDS,BOATRENTAL,CANOE,DISCGOLF,DOGPARK,ENVCTR,FISHING,GREENWAYACCESS,GYM,MULTIPURPOSEFIELD,OUTDOORBASKETBALL,PICNICSHELTER,PLAYGROUND,POOL,COMMUNITYCENTER,NEIGHBORHOODCENTER,TENNISCOURTS,TRACK,WALKINGTRAILS,JURISDICTION,RESTROOMS,AMUSEMENTTRAIN,CAROUSEL,TENNISCENTER,THEATER,BOCCE,HANDBALL,HORSESHOE,INLINESKATING,SANDVOLLEYBALL,SKATEPARK,ACTIVE_ADULT,BMXTRACK,BOATRIDE,LIBRARY,MUSEUM,TEEN,BIKING,LIVEANIMALS,GARDENS,EQUESTRIAN
0,Anderson Point Park,"20 Anderson Point Dr, Raleigh, NC",35.770887,-78.542011,No,No,No,Yes,No,No,No,No,Yes,No,No,No,Yes,Yes,No,No,No,No,No,Yes,City of Raleigh,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN
1,"Annie Louise Wilkerson, MD Nature Preserve Park","11408 Raven Ridge Rd, Raleigh, NC",35.922096,-78.598837,No,No,No,No,No,No,Yes,No,No,No,No,No,Yes,No,No,No,No,No,No,Yes,City of Raleigh,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN


In [377]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 46 columns):
NAME                  263 non-null object
ADDRESS               262 non-null object
Lat                   218 non-null float64
Lon                   218 non-null float64
ARTSCENTER            254 non-null object
BALLFIELDS            255 non-null object
BOATRENTAL            254 non-null object
CANOE                 259 non-null object
DISCGOLF              254 non-null object
DOGPARK               254 non-null object
ENVCTR                254 non-null object
FISHING               255 non-null object
GREENWAYACCESS        253 non-null object
GYM                   255 non-null object
MULTIPURPOSEFIELD     255 non-null object
OUTDOORBASKETBALL     255 non-null object
PICNICSHELTER         258 non-null object
PLAYGROUND            257 non-null object
POOL                  254 non-null object
COMMUNITYCENTER       254 non-null object
NEIGHBORHOODCENTER    254 non-null object
TENNISCOURT

Latitude and Longitude coordinates have some missing values. Those values must be filled in order to use mapping for our purposes. Missing values have no impact in other variables of our table. They are missing because the facility does not provide them

In [378]:
missing=table[table['Lat'].isna()]
missing

,NAME,ADDRESS,Lat,Lon,ARTSCENTER,BALLFIELDS,BOATRENTAL,CANOE,DISCGOLF,DOGPARK,ENVCTR,FISHING,GREENWAYACCESS,GYM,MULTIPURPOSEFIELD,OUTDOORBASKETBALL,PICNICSHELTER,PLAYGROUND,POOL,COMMUNITYCENTER,NEIGHBORHOODCENTER,TENNISCOURTS,TRACK,WALKINGTRAILS,JURISDICTION,RESTROOMS,AMUSEMENTTRAIN,CAROUSEL,TENNISCENTER,THEATER,BOCCE,HANDBALL,HORSESHOE,INLINESKATING,SANDVOLLEYBALL,SKATEPARK,ACTIVE_ADULT,BMXTRACK,BOATRIDE,LIBRARY,MUSEUM,TEEN,BIKING,LIVEANIMALS,GARDENS,EQUESTRIAN
212,American Tobacco Trail - White Oak,"1305 White Oak Church Road, Apex, NC 27523",NaN,NaN,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,Yes,Wake County,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,Yes
213,Robertson Millpond Preserve,"6333 Robertson Pond Road, Wendell, NC 27591",NaN,NaN,No,No,Yes,Yes,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,Wake County,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No
214,Jack Smith Park,"9725 Penny Road, Cary, NC",NaN,NaN,No,No,No,No,No,Yes,No,No,Yes,No,No,No,Yes,Yes,No,No,No,No,No,No,Town of Cary,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN
215,New Hope Church Road Trailhead Park,"2575 New Hope Church Road, Cary, NC",NaN,NaN,No,No,No,No,No,No,No,No,Yes,No,Yes,No,Yes,No,No,No,No,No,No,No,Town of Cary,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN
216,Good Hope Farm,"1580 Morrisville Carpenter Road, Cary, NC",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Town of Cary,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN
217,Carpenter Park,"4420 Louis Stephens Drive, Cary, NC",NaN,NaN,No,No,No,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes,No,No,No,Yes,No,Yes,Town of Cary,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN
218,Symphony Lake Park,NaN,NaN,NaN,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,Yes,Town of Cary,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN
219,Meeting Place Park,"601 Kildaire Farm Road, Cary, NC",NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,Town of Cary,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN
220,Downtown Park,"319 S. Academy Street, Cary, NC",NaN,NaN,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Town of Cary,No,No,No,No,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN
221,Veterans Freedom Park,"1517 N Harrison Ave, Cary, NC",NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Town of Cary,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN


In [379]:
table[table['ADDRESS'].isna()]

,NAME,ADDRESS,Lat,Lon,ARTSCENTER,BALLFIELDS,BOATRENTAL,CANOE,DISCGOLF,DOGPARK,ENVCTR,FISHING,GREENWAYACCESS,GYM,MULTIPURPOSEFIELD,OUTDOORBASKETBALL,PICNICSHELTER,PLAYGROUND,POOL,COMMUNITYCENTER,NEIGHBORHOODCENTER,TENNISCOURTS,TRACK,WALKINGTRAILS,JURISDICTION,RESTROOMS,AMUSEMENTTRAIN,CAROUSEL,TENNISCENTER,THEATER,BOCCE,HANDBALL,HORSESHOE,INLINESKATING,SANDVOLLEYBALL,SKATEPARK,ACTIVE_ADULT,BMXTRACK,BOATRIDE,LIBRARY,MUSEUM,TEEN,BIKING,LIVEANIMALS,GARDENS,EQUESTRIAN
218,Symphony Lake Park,NaN,NaN,NaN,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,Yes,Town of Cary,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN


Upon further inspection, record 218 has no address and coordinates data. Fortunately we can use the name of the park to fill
the missing value with geopy

In [380]:
#The address is the name of the park
address = 'Symphony Lake Park'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Symphony Lake Park are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Symphony Lake Park are 35.727358, -78.7969536725968.


In [381]:
table.loc[218,'Lat']=latitude


In [382]:
table.loc[218,'Lon']=longitude
table.loc[218,'ADDRESS']=location.address

In [383]:
table.loc[218][0:5]

NAME                                         Symphony Lake Park
ADDRESS       Symphony Lake, Cary, Wake County, North Caroli...
Lat                                                     35.7274
Lon                                                     -78.797
ARTSCENTER                                                   No
Name: 218, dtype: object

Latitude and Longitude has been properly filed. Address has been added for convenience

In [384]:
table1=table

Fortunately the addresses are available for missing Longitude and latitude coordinates. We will use those addresses to fill the missing Lon and Lat values. We loop through the Longitude and Latitude using geopy

In [361]:
address = 'Symphony Lake Park'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 35.727358, -78.7969536725968.


In [366]:
#for lat,lon,address in zip(table1['Lat'],table1['Lon'],table1['ADDRESS']):
 #   if lat == 'NaN':
#        location = geolocator.geocode(address)
#        latitude = location.latitude
        
        
       

In [385]:
table1.head(2)

,NAME,ADDRESS,Lat,Lon,ARTSCENTER,BALLFIELDS,BOATRENTAL,CANOE,DISCGOLF,DOGPARK,ENVCTR,FISHING,GREENWAYACCESS,GYM,MULTIPURPOSEFIELD,OUTDOORBASKETBALL,PICNICSHELTER,PLAYGROUND,POOL,COMMUNITYCENTER,NEIGHBORHOODCENTER,TENNISCOURTS,TRACK,WALKINGTRAILS,JURISDICTION,RESTROOMS,AMUSEMENTTRAIN,CAROUSEL,TENNISCENTER,THEATER,BOCCE,HANDBALL,HORSESHOE,INLINESKATING,SANDVOLLEYBALL,SKATEPARK,ACTIVE_ADULT,BMXTRACK,BOATRIDE,LIBRARY,MUSEUM,TEEN,BIKING,LIVEANIMALS,GARDENS,EQUESTRIAN
0,Anderson Point Park,"20 Anderson Point Dr, Raleigh, NC",35.770887,-78.542011,No,No,No,Yes,No,No,No,No,Yes,No,No,No,Yes,Yes,No,No,No,No,No,Yes,City of Raleigh,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN
1,"Annie Louise Wilkerson, MD Nature Preserve Park","11408 Raven Ridge Rd, Raleigh, NC",35.922096,-78.598837,No,No,No,No,No,No,Yes,No,No,No,No,No,Yes,No,No,No,No,No,No,Yes,City of Raleigh,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN


Alternate way to get the goordinates. Looping through the dataframe is not reliable due to errors. The following steps are meant to simplify the process

In [390]:
nan=table1.loc[table1['Lat'].isna(),'ADDRESS']
nan=list(nan)
nan

['1305 White Oak Church Road, Apex, NC 27523',
 '6333\xa0Robertson\xa0Pond Road, Wendell, NC 27591',
 '9725 Penny Road, Cary, NC',
 '2575 New Hope Church Road, Cary, NC',
 '1580 Morrisville Carpenter Road, Cary, NC',
 '4420 Louis Stephens Drive, Cary, NC',
 '601 Kildaire Farm Road, Cary, NC',
 '319 S. Academy Street, Cary, NC',
 '1517 N Harrison Ave, Cary, NC',
 '2139 Old Reedy Creek Road, Cary, NC',
 '9950 Durant Road, Raleigh, NC',
 '2100 Old NC Hwy 98, Wake Forest, NC',
 '107 Pullen Road, Raleigh, NC',
 '5611 Jaguar Park Dr, Raleigh, NC',
 '1256 McCauley St, Raleigh, NC',
 '416 E Lee St, Raleigh, NC',
 '131 South Fuquay Avenue, Fuquay-Varina, NC',
 '221 E Garner Rd, Garner, NC',
 '810 North First Avenue, Knightdale, NC',
 '207 Main St, Knightdale, NC',
 '1385 N Smithfield Rd, Knightdale, NC',
 '1137 Old Knight Rd, Knightdale, NC',
 '100 Parkside Commons Drive, Knightdale, NC',
 '200 South Main St, Rolesville, NC',
 '425 Nature Park Drive, Wake Forest, NC',
 '121 Redford Place Drive,

In [397]:
len(nan)

44

Much easier to loop through a list

In [409]:
geolocator = Nominatim(user_agent="ny_explorer")
Lat=[]
Lon=[]
for each in nan:
    try:
        
        address = each
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        Lat.append(latitude)
        Lon.append(longitude)
    except:
        Lat.append(latitude)
        Lon.append(longitude)
        
   
    

In [413]:
print(len(Lon))
print(len(Lat))

44
44


Both our Lon and Lat lists have the same size with the nan list.

In [417]:
table1.loc[table1['Lat'].isna(),'Lat']=Lat
table1.loc[table1['Lon'].isna(),'Lon']=Lon

In [419]:
table1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 46 columns):
NAME                  263 non-null object
ADDRESS               263 non-null object
Lat                   263 non-null float64
Lon                   263 non-null float64
ARTSCENTER            254 non-null object
BALLFIELDS            255 non-null object
BOATRENTAL            254 non-null object
CANOE                 259 non-null object
DISCGOLF              254 non-null object
DOGPARK               254 non-null object
ENVCTR                254 non-null object
FISHING               255 non-null object
GREENWAYACCESS        253 non-null object
GYM                   255 non-null object
MULTIPURPOSEFIELD     255 non-null object
OUTDOORBASKETBALL     255 non-null object
PICNICSHELTER         258 non-null object
PLAYGROUND            257 non-null object
POOL                  254 non-null object
COMMUNITYCENTER       254 non-null object
NEIGHBORHOODCENTER    254 non-null object
TENNISCOURT

Both Latitude and Longitude values are non null.

In [420]:
table1.to_csv('Wake_countyParks',index=False)